In [1]:
import numpy as np
from numpy.random import seed
from numpy import random

import pandas as pd
from sklearn.decomposition import NMF

In [2]:
ratings = pd.read_csv('data/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
# movie_id_with_titles = pd.read_csv('data/df_movieId_title.csv')
# movie_id_with_titles.head()

In [4]:
# ratings_add = pd.merge(ratings, movie_id_with_titles, how="left", on=["movieId"])
# ratings_add = ratings_add[['userId', 'movieId', 'rating', 'title', 'title_only']]
# ratings_add.head()
# ratings_add.shape # (100836, 6)
# ratings_add.drop_duplicates()
# ratings_add.set_index(['userId', 'title_only']).unstack()['rating']

In [5]:
wide = ratings.pivot(index = 'userId', columns = 'movieId', values = 'rating')
wide.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
wide.fillna(wide.median(), inplace = True)
wide.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,3.5,4.0,3.0,3.0,4.0,3.0,3.0,3.0,3.5,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
2,4.0,3.5,3.0,3.0,3.0,4.0,3.0,3.0,3.0,3.5,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
3,4.0,3.5,3.0,3.0,3.0,4.0,3.0,3.0,3.0,3.5,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
4,4.0,3.5,3.0,3.0,3.0,4.0,3.0,3.0,3.0,3.5,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
5,4.0,3.5,3.0,3.0,3.0,4.0,3.0,3.0,3.0,3.5,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0


In [7]:
nmf = NMF(n_components = 2, random_state = 666, max_iter = 20000) ## more iterations needed, doesn't converge 10000 takea a bit longer but converges
nmf.fit(wide)

/home/mmuratardag/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 20000 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


NMF(max_iter=20000, n_components=21, random_state=666)

In [8]:
Q = nmf.components_

In [9]:
# movieIDs = list(wide.columns)
# [print(a, b) for (a, b) in zip(Q.T, movieIDs)]

In [10]:
seed(666)
new_user_rating = random.randint(5, size=(1, 9724)) # doesn't make any sense to generate a row with 9724 ratings -- just for the sake of exercise completion
P = nmf.transform(new_user_rating)

In [11]:
predictions = np.dot(P, Q)

In [12]:
recommendations = pd.DataFrame(predictions, columns=wide.columns)

In [13]:
items_to_recommend = recommendations.T
items_to_recommend.columns = ['predicted_rating']
items_to_recommend.sort_values(by='predicted_rating', ascending=False)[:3]

,predicted_rating
movieId,
31364,2.845421
85,2.843419
70946,2.833283


In [14]:
df = pd.read_csv('data/df.csv', na_values=[' '])
# df[df['movieId'] == 356]['title'].unique() # Forrest Gump
# df[df['movieId'] == 2858]['title'].unique() # American Beauty
# df[df['movieId'] == 85]['title'].unique() # Angels and Insects

# df[df['movieId'] == 31364]['title'].unique() # Memories of Murder (Salinui chueok)
# df[df['movieId'] == 70946]['title'].unique() # Troll 2